In [1]:
import csv

import geopandas as gpd
import pandas as pd
import requests
from psycopg2.pool import ThreadedConnectionPool
from shapely.geometry import shape

In [2]:
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f.readlines():
        if line.startswith("#"):
            continue
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

postgres_thread_pool = ThreadedConnectionPool(
    1,
    50,
    host=env["API_POSTGRES_HOST"],
    port=env["API_POSTGRES_PORT"],
    user=env["API_POSTGRES_USERNAME"],
    password=env["API_POSTGRES_PASSWORD"],
)

In [3]:
CSV_URL = "https://hdr.undp.org/sites/default/files/data/2020/IHDI_HDR2020_040722.csv"
YEAR = 2019

In [4]:
with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode("utf-8")

csv_rows = list(csv.reader(decoded_content.splitlines(), delimiter=","))
df = pd.DataFrame(csv_rows[1:], columns=csv_rows[0])
df.head()

,iso3,country,hdicode,region,hdi_2010,hdi_2011,hdi_2012,hdi_2013,hdi_2014,hdi_2015,...,ineq_inc_2010,ineq_inc_2011,ineq_inc_2012,ineq_inc_2013,ineq_inc_2014,ineq_inc_2015,ineq_inc_2016,ineq_inc_2017,ineq_inc_2018,ineq_inc_2019
0,AFG,Afghanistan,Low,SA,0.472,0.477,0.489,0.496,0.5,0.5,...,,,,10.8,10.8,10.8,10.8,10.8,,
1,AGO,Angola,Medium,SSA,0.517,0.533,0.544,0.555,0.565,0.572,...,36.4,50,50,50,28.9,28.9,28.9,28.9,28.9,28.9
2,ALB,Albania,High,ECA,0.745,0.764,0.775,0.782,0.787,0.788,...,14.4,18.3,18.3,18.3,18.3,18.3,18.3,12.2,13.2,13.17898
3,AND,Andorra,Very High,,0.837,0.836,0.858,0.856,0.863,0.862,...,,,,,,,,,,
4,ARE,United Arab Emirates,Very High,AS,0.82,0.826,0.832,0.838,0.847,0.859,...,,,,,,,,,,


In [5]:
cols = f"iso3 country hdicode region hdi_{YEAR}".split()
df = df.loc[:, cols]  # filter columns
df["hdi_2019"] = pd.to_numeric(df["hdi_2019"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   iso3      206 non-null    object 
 1   country   206 non-null    object 
 2   hdicode   206 non-null    object 
 3   region    206 non-null    object 
 4   hdi_2019  200 non-null    float64
dtypes: float64(1), object(4)
memory usage: 8.2+ KB


In [61]:
df

,iso3,country,hdicode,region,hdi_2019
0,AFG,Afghanistan,Low,SA,0.511
1,AGO,Angola,Medium,SSA,0.581
2,ALB,Albania,High,ECA,0.795
3,AND,Andorra,Very High,,0.868
4,ARE,United Arab Emirates,Very High,AS,0.890
...,...,...,...,...,...
201,,Europe and Central Asia,,,0.791
202,,Latin America and the Caribbean,,,0.766
203,,South Asia,,,0.641
204,,Sub-Saharan Africa,,,0.547


In [85]:
# get h3index from iso3 code
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()


def first_query(table_name: str, country_iso3_code: str, hdi_value: float) -> str:
    """Makes a sql query to create a table.

    The table contains the h3 indexes of country code and a hdi column with the given value
    """

    query = f"""
        drop table if exists {table_name};
        create table {table_name} (h3index, hdi)
        as
        select
            h3_uncompact(gr."h3Compact"::h3index[], 6) as h3index,
            {hdi_value} as hdi
        from geo_region gr
        join admin_region ar on gr.id = ar."geoRegionId"
        where ar."isoA3"  = '{country_iso3_code}';
        """
    return query


def insert_rest_query(table_name: str, country_iso3_code: str, hdi_value: float) -> str:

    query = f"""
        insert into {table_name} (h3index, hdi)
        select
            h3_uncompact(gr."h3Compact"::h3index[], 6) as h3index,
            {hdi_value} as hdi
        from geo_region gr
        join admin_region ar on gr.id = ar."geoRegionId"
        where ar."isoA3"  = '{country_iso3_code}';
        """
    return query

In [86]:
print(first_query("h3_grid_hdi_global", "AFG", 99.7))


        drop table if exists h3_grid_hdi_global;
        create table h3_grid_hdi_global (h3index, hdi)
        as
        select
            h3_uncompact(gr."h3Compact"::h3index[], 6) as h3index,
            99.7 as hdi
        from geo_region gr
        join admin_region ar on gr.id = ar."geoRegionId"
        where ar."isoA3"  = 'AFG';
        


In [87]:
print(insert_rest_query("h3_grid_hdi_global", "ARG", 0.1))


        insert into h3_grid_hdi_global (h3index, hdi)
        select
            h3_uncompact(gr."h3Compact"::h3index[], 6) as h3index,
            0.1 as hdi
        from geo_region gr
        join admin_region ar on gr.id = ar."geoRegionId"
        where ar."isoA3"  = 'ARG';
        
